In [43]:
import pandas as pd
import os
import numpy as np

# Read the CSV and Perform Basic Data Cleaning

In [44]:
df = pd.read_csv(os.path.join("..","Resources","exoplanet_data.csv"))
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
koi = df.dropna()
print(f"Number rows: {len(df)}")
koi.head()

Number rows: 6991


,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [45]:
# koi = pd.get_dummies(df,drop_first=True)
# koi.head()

In [46]:
# Set features. This will also be used as your x values.
selected_features = ['koi_impact', 'koi_depth', 'koi_prad', 'koi_teq',\
    'koi_steff','koi_slogg','koi_srad']

#data = df[selected_features]
data = df.drop(columns=['koi_disposition'])

feature_names = data.columns
data.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [47]:
target_names = ['CANDIDATE', 'FALSE POSITIVE']
target = koi.koi_disposition#_CONFIRMED.values.reshape(-1, 1)
target

0            CONFIRMED
1       FALSE POSITIVE
2       FALSE POSITIVE
3            CONFIRMED
4            CONFIRMED
             ...      
6986    FALSE POSITIVE
6987    FALSE POSITIVE
6988         CANDIDATE
6989    FALSE POSITIVE
6990    FALSE POSITIVE
Name: koi_disposition, Length: 6991, dtype: object

# Create a Train Test Split

Use `koi_disposition` for the y values

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=7)

In [49]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(target)
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [50]:
# Scale your data
from sklearn.preprocessing import StandardScaler,MinMaxScaler

# Create a StandardScater model and fit it to the training data
X_scaler = MinMaxScaler().fit(X_train)

# Transform the training and testing data using the X_scaler and y_scaler models
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model



In [51]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=500)
model.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=500)

In [52]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.8514209422086592
Testing Data Score: 0.8335240274599542


Feature Selection

In [53]:
## Recursive Feature Elimination
from sklearn.feature_selection import RFE
# n_features_to_select=20 because of the PCA Analisis
selector = RFE(model, n_features_to_select=20, step=1)
selector = selector.fit(X_train_scaled, y_train_encoded)
topFeatures = data.loc[:,selector.support_]
topFeatures.columns

Index(['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec',
       'koi_period', 'koi_period_err1', 'koi_period_err2', 'koi_time0bk_err1',
       'koi_time0bk_err2', 'koi_impact_err1', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_teq',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff_err1', 'koi_slogg',
       'koi_slogg_err2'],
      dtype='object')

In [54]:
X_train, X_test, y_train, y_test = train_test_split(topFeatures, target, random_state=7)

In [55]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [56]:
PCA_Model = LogisticRegression(max_iter=500)

## Fit model
PCA_Model.fit(X_train_scaled, y_train_encoded)

LogisticRegression(max_iter=500)

In [57]:
print(f"Training Data Score: {PCA_Model.score(X_train_scaled, y_train_encoded)}")
print(f"Testing Data Score: {PCA_Model.score(X_test_scaled, y_test_encoded)}")

Training Data Score: 0.8495136372305931
Testing Data Score: 0.8260869565217391


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [58]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV

param_grid = {'penalty': ['l1', 'l2'],
              'C': np.logspace(-5, 5, 10),
              'solver': ['liblinear']}

grid = GridSearchCV(PCA_Model, param_grid, verbose=3)

In [59]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train_encoded)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END C=1e-05, penalty=l1, solver=liblinear;, score=0.238 total time=   0.0s
[CV 2/5] END C=1e-05, penalty=l1, solver=liblinear;, score=0.238 total time=   0.0s
[CV 3/5] END C=1e-05, penalty=l1, solver=liblinear;, score=0.238 total time=   0.0s
[CV 4/5] END C=1e-05, penalty=l1, solver=liblinear;, score=0.238 total time=   0.0s
[CV 5/5] END C=1e-05, penalty=l1, solver=liblinear;, score=0.238 total time=   0.0s
[CV 1/5] END C=1e-05, penalty=l2, solver=liblinear;, score=0.503 total time=   0.0s
[CV 2/5] END C=1e-05, penalty=l2, solver=liblinear;, score=0.503 total time=   0.0s
[CV 3/5] END C=1e-05, penalty=l2, solver=liblinear;, score=0.502 total time=   0.0s
[CV 4/5] END C=1e-05, penalty=l2, solver=liblinear;, score=0.503 total time=   0.0s
[CV 5/5] END C=1e-05, penalty=l2, solver=liblinear;, score=0.503 total time=   0.0s
[CV 1/5] END C=0.0001291549665014884, penalty=l1, solver=liblinear;, score=0.238 total time=   0.0

GridSearchCV(estimator=LogisticRegression(max_iter=500),
             param_grid={'C': array([1.00000000e-05, 1.29154967e-04, 1.66810054e-03, 2.15443469e-02,
       2.78255940e-01, 3.59381366e+00, 4.64158883e+01, 5.99484250e+02,
       7.74263683e+03, 1.00000000e+05]),
                         'penalty': ['l1', 'l2'], 'solver': ['liblinear']},
             verbose=3)

In [61]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 7742.636826811277, 'penalty': 'l2', 'solver': 'liblinear'}
0.8823208581055022


In [63]:
hpModel = LogisticRegression(**grid.best_params_, max_iter=500)

hpModel.fit(X_train_scaled, y_train_encoded)
print(f"Training Data Score: {hpModel.score(X_train_scaled, y_train_encoded)}")
print(f"Testing Data Score: {hpModel.score(X_test_scaled, y_test_encoded)}")

Training Data Score: 0.8823192828533283
Testing Data Score: 0.8758581235697941


# Save the Model

In [64]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'LogisticRegression.sav'
joblib.dump(hpModel, filename)

['LogisticRegression.sav']